<a href="https://colab.research.google.com/github/dangzzmedina/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-06 19:51:24--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.02MB/s    in 0.2s    

2021-09-06 19:51:25 (6.02 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### **Deliverable 2 Vine Paid/Unpaid**

In [21]:
vine_df = df.select(['review_id', 'helpful_votes', 'total_votes', 'vine', 'star_rating'])
vine_df.show()

+--------------+-------------+-----------+----+-----------+
|     review_id|helpful_votes|total_votes|vine|star_rating|
+--------------+-------------+-----------+----+-----------+
| RTIS3L2M1F5SM|            0|          0|   N|          5|
| R1ZV7R40OLHKD|            0|          0|   N|          5|
|R3BH071QLH8QMC|            0|          1|   N|          1|
|R127K9NTSXA2YH|            0|          0|   N|          3|
|R32ZWUXDJPW27Q|            0|          0|   N|          4|
|R3AQQ4YUKJWBA6|            0|          0|   N|          1|
|R2F0POU5K6F73F|            0|          0|   N|          5|
|R3VNR804HYSMR6|            0|          0|   N|          5|
| R3GZTM72WA2QH|            0|          0|   N|          5|
| RNQOY62705W1K|            0|          0|   N|          4|
|R1VTIA3JTYBY02|            0|          0|   N|          5|
|R29DOU8791QZL8|            0|          0|   N|          1|
|R15DUT1VIJ9RJZ|            0|          0|   N|          2|
|R3IMF2MQ3OU9ZM|            0|          

In [22]:
#Total Votes greater than 20
votes_20_df = vine_df.filter('total_votes >=20')
votes_20_df.show()

+--------------+-------------+-----------+----+-----------+
|     review_id|helpful_votes|total_votes|vine|star_rating|
+--------------+-------------+-----------+----+-----------+
| R4PKAZRQJJX14|           21|         34|   N|          1|
|R2CI0Y288CC7E2|           21|         35|   N|          1|
|R127WEQY2FM1T3|          147|        175|   N|          1|
|R3EZ0EPYLDA34S|           14|         31|   N|          1|
|R2FJ94555FZH32|           55|         60|   N|          2|
|R1U3AR67RE273L|           51|         65|   N|          1|
|R3PZOXA5X1U8KW|           31|         36|   N|          4|
| R6KTC1OPIOIIG|           19|         34|   N|          2|
|R36O341WWXXKNP|           28|         31|   N|          5|
|R3GSK9MM8DNOYI|            4|         32|   N|          1|
|R10LZVBLQHBVJ0|          151|        198|   N|          2|
|R1VR5GLGY1GE7N|           49|         51|   N|          1|
|R3GUUC6NT4QMCL|            5|         36|   N|          1|
|R1ZSD020RT18I5|            7|        13

In [23]:
#Votes greater than 50% when dividing helpful votes and total votes
votes_50_df = votes_20_df.filter(votes_20_df['helpful_votes']/votes_20_df['total_votes']>=0.5)
votes_50_df.show()

+--------------+-------------+-----------+----+-----------+
|     review_id|helpful_votes|total_votes|vine|star_rating|
+--------------+-------------+-----------+----+-----------+
| R4PKAZRQJJX14|           21|         34|   N|          1|
|R2CI0Y288CC7E2|           21|         35|   N|          1|
|R127WEQY2FM1T3|          147|        175|   N|          1|
|R2FJ94555FZH32|           55|         60|   N|          2|
|R1U3AR67RE273L|           51|         65|   N|          1|
|R3PZOXA5X1U8KW|           31|         36|   N|          4|
| R6KTC1OPIOIIG|           19|         34|   N|          2|
|R36O341WWXXKNP|           28|         31|   N|          5|
|R10LZVBLQHBVJ0|          151|        198|   N|          2|
|R1VR5GLGY1GE7N|           49|         51|   N|          1|
|R2OWDAR0EDEPTF|           23|         28|   N|          1|
|R29BOS5HMAY1LO|           88|        110|   N|          5|
| R8WD0136CETA7|           99|        119|   N|          2|
|R16621F16PQN5A|           24|         3

In [24]:
# Vine Review Program = Yes
vine_votes_df = votes_50_df.filter(votes_50_df['vine'] == 'Y')
vine_votes_df.show()

+--------------+-------------+-----------+----+-----------+
|     review_id|helpful_votes|total_votes|vine|star_rating|
+--------------+-------------+-----------+----+-----------+
|R3KKUSGFZWSUIY|           56|         63|   Y|          5|
|R10FO5UKKVZBK2|           23|         23|   Y|          3|
| RM4KSGEOR7MU1|           19|         24|   Y|          5|
| RG7VRMYLEXD23|           22|         26|   Y|          4|
|R11O4YSCPSNL6L|           20|         26|   Y|          3|
|R286MFBAJ8NPD6|           46|         51|   Y|          5|
|R1JRR530H4COA2|           22|         28|   Y|          5|
| RQ5WD90PUNBU9|           21|         24|   Y|          5|
|R12648VHCQWUV9|           21|         28|   Y|          4|
|R3KAW29CJ8L6DQ|           17|         20|   Y|          5|
|R1OWK33OPI45KT|           55|         58|   Y|          5|
|R2UP7VTED8O425|           35|         41|   Y|          5|
|R3N4QAXTQCA1ED|           18|         20|   Y|          5|
|R3EPHK0JGQGE91|           21|         2

In [25]:
# Vine Review Program = No
novine_votes_df = votes_50_df.filter(votes_50_df['vine'] == 'N')
novine_votes_df.show()

+--------------+-------------+-----------+----+-----------+
|     review_id|helpful_votes|total_votes|vine|star_rating|
+--------------+-------------+-----------+----+-----------+
| R4PKAZRQJJX14|           21|         34|   N|          1|
|R2CI0Y288CC7E2|           21|         35|   N|          1|
|R127WEQY2FM1T3|          147|        175|   N|          1|
|R2FJ94555FZH32|           55|         60|   N|          2|
|R1U3AR67RE273L|           51|         65|   N|          1|
|R3PZOXA5X1U8KW|           31|         36|   N|          4|
| R6KTC1OPIOIIG|           19|         34|   N|          2|
|R36O341WWXXKNP|           28|         31|   N|          5|
|R10LZVBLQHBVJ0|          151|        198|   N|          2|
|R1VR5GLGY1GE7N|           49|         51|   N|          1|
|R2OWDAR0EDEPTF|           23|         28|   N|          1|
|R29BOS5HMAY1LO|           88|        110|   N|          5|
| R8WD0136CETA7|           99|        119|   N|          2|
|R16621F16PQN5A|           24|         3

In [26]:
#Paid Vote Counts

paid_vote_count = vine_votes_df.count()
paid_vote_count

94

In [27]:
#Not Paid Vote Counts

not_paid_vote_count = novine_votes_df.count()
not_paid_vote_count

40471

In [30]:
# 5 star reviews = Vine -> Yes
fivestar_paid_reviews = vine_votes_df.filter(vine_votes_df['star_rating']== 5) 
fivestar_paid_reviews.count()

48

In [32]:
# 5 star reviews = Vine -> No
fivestar_unpaid_reviews = novine_votes_df.filter(novine_votes_df['star_rating']== 5)
fivestar_unpaid_reviews.count()

15663

In [35]:
#Percentage 5 star reviews Vine -> Yes

fivestar_percentage_total = fivestar_paid_reviews.count()/paid_vote_count
fivestar_percentage_total

0.5106382978723404

In [37]:
#Percentage 5 star reviews Vine -> No

unpaid_fivestar_percentage_total = fivestar_unpaid_reviews.count()/not_paid_vote_count
unpaid_fivestar_percentage_total

0.38701786464381904